# コードのテスト
## prepare_data.py の関数について
- ラベル情報などが書いたcsvファイル（tox21_compoundData.csv）からTox21の画像のパスとラベル(y_train,y_valid,y_test)を取得する

In [1]:
import os
gpu_num = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

# モジュールimport
import os, sys
current_dir = os.path.dirname(os.path.abspath("__file__"))
path = os.path.join(current_dir, '../../')
sys.path.append(path)
from dataset import plot_log, prepare_data
from transformer import get_train_valid_test
from model import define_model, multi_loss, my_callback, my_metric
from predicter import roc_curve, conf_matrix, multi_predict, grad_cam

import numpy as np
import keras
import keras.backend as K
import matplotlib.pyplot as plt

sys.path.append(r'/home/tmp10014/jupyterhub/notebook/other/lib_DL/mixup-generator')
from random_eraser import get_random_eraser

plot_log.py: loaded as module file
prepare_data.py: loaded as module file


Using TensorFlow backend.
/home/tmp10014/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


get_train_valid_test.py: loaded as module file
define_model.py: loaded as module file
multi_loss.py: loaded as module file
my_callback.py: loaded as module file
my_metric.py: loaded as module file
roc_curve.py: loaded as module file
conf_matrix.py: loaded as module file
multi_predict.py: loaded as module file
grad_cam.py: loaded as module file


## prepare_data.make_label_df

In [2]:
# ラベル情報などが書いたcsvファイル（tox21_compoundData.csv）
csv_path = r'../../storage/tox21_compoundData.csv'
# IDにファイルのフルパスつける
img_dir = r'../../storage/img/Wash/331x331/'
# 画像ファイルのサフィックス
img_suffix = r'_000.jpg'
# 欠損値を-1に置換したtox21_compoundData.csvのデータフレーム
df = prepare_data.make_label_df(csv_path, img_dir, img_suffix=img_suffix)
df.head()

,ID,inchikey,sdftitle,order,set,CVfold,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
0,../../storage/img/Wash/331x331/train_all/NCGC0...,PEJLNXHANOHNSU-UHFFFAOYSA-N,NCGC00178831-03,0,training,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
1,../../storage/img/Wash/331x331/train_all/NCGC0...,SEACYXSIPDVVMV-UHFFFAOYSA-L,NCGC00166114-03,1,training,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
2,../../storage/img/Wash/331x331/train_all/NCGC0...,WQGJEAMPBSZCIF-VEKNOCPUSA-N,NCGC00263563-01,2,training,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0
3,../../storage/img/Wash/331x331/train_all/NCGC0...,CNYGFPPAGUCRIC-UHFFFAOYSA-L,NCGC00013058-02,3,training,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0
4,../../storage/img/Wash/331x331/train_all/NCGC0...,LYCYLGFSIXIXAB-NUZRHMIVSA-N,NCGC00167516-01,4,training,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## prepare_data.make_train_val_test

In [3]:
# Tox21の画像のパスとラベル(y_train,y_valid,y_test)を取得する
train_files, validation_files, test_files, y_train, y_valid, y_test = prepare_data.make_train_val_test(df)

print('train_files[0:5]:\n', train_files[0:5])
print('validation_files[0:5]:\n', validation_files[0:5])
print('test_files[0:5]:\n', test_files[0:5])
print('y_train[0:5]:\n', y_train[0:5])
print('y_valid[0:5]:\n', y_valid[0:5])
print('y_test[0:5]:\n', y_test[0:5])

training_set_y.shape: (11764, 12)
validation_set_y.shape: (296, 12)
test_set_y.shape: (647, 12)
train_files[0:5]:
 0    ../../storage/img/Wash/331x331/train_all/NCGC0...
1    ../../storage/img/Wash/331x331/train_all/NCGC0...
2    ../../storage/img/Wash/331x331/train_all/NCGC0...
3    ../../storage/img/Wash/331x331/train_all/NCGC0...
4    ../../storage/img/Wash/331x331/train_all/NCGC0...
Name: ID, dtype: object
validation_files[0:5]:
 11764    ../../storage/img/Wash/331x331/test/NCGC002614...
11765    ../../storage/img/Wash/331x331/test/NCGC002616...
11766    ../../storage/img/Wash/331x331/test/NCGC002609...
11767    ../../storage/img/Wash/331x331/test/NCGC002612...
11768    ../../storage/img/Wash/331x331/test/NCGC002618...
Name: ID, dtype: object
test_files[0:5]:
 12060    ../../storage/img/Wash/331x331/Final_Evaluatio...
12061    ../../storage/img/Wash/331x331/Final_Evaluatio...
12062    ../../storage/img/Wash/331x331/Final_Evaluatio...
12063    ../../storage/img/Wash/331x331/Final_Ev

## prepare_data.make_train_val_test_CVfold

In [4]:
# CV(Cross Validation)の画像ファイルパスとラベルを取得する
for i in range(6):
    print('\n----- CV:',i, '-----')
    train_files, y_train, validation_files, y_valid, test_files, y_test = prepare_data.make_train_val_test_CVfold(df
                                                                                                                  , valid_cv=i
                                                                                                                  , not_set_name='test')

    print('\ntrain_files[0:5]:\n', train_files[0:5])
    print('validation_files[0:5]:\n', validation_files[0:5])
    print('test_files[0:5]:\n', test_files[0:5])
    print('y_train[0:5]:\n', y_train[0:5])
    print('y_valid[0:5]:\n', y_valid[0:5])
    print('y_test[0:5]:\n', y_test[0:5])


----- CV: 0 -----
[1, 2, 3, 4, 5]_cv_train_y.shape: (6666, 12)
0_cv_valid_y.shape: (1558, 12)
test_set_y.shape: (647, 12)

train_files[0:5]:
 0     ../../storage/img/Wash/331x331/train_all/NCGC0...
2     ../../storage/img/Wash/331x331/train_all/NCGC0...
25    ../../storage/img/Wash/331x331/train_all/NCGC0...
36    ../../storage/img/Wash/331x331/train_all/NCGC0...
61    ../../storage/img/Wash/331x331/train_all/NCGC0...
Name: ID, dtype: object
validation_files[0:5]:
 21     ../../storage/img/Wash/331x331/train_all/NCGC0...
47     ../../storage/img/Wash/331x331/train_all/NCGC0...
67     ../../storage/img/Wash/331x331/train_all/NCGC0...
69     ../../storage/img/Wash/331x331/train_all/NCGC0...
101    ../../storage/img/Wash/331x331/train_all/NCGC0...
Name: ID, dtype: object
test_files[0:5]:
 12060    ../../storage/img/Wash/331x331/Final_Evaluatio...
12061    ../../storage/img/Wash/331x331/Final_Evaluatio...
12062    ../../storage/img/Wash/331x331/Final_Evaluatio...
12063    ../../storage/im

## prepare_data.make_label_df_NoNA

In [5]:
# ラベルの欠損値もつレコード削除したtox21_compoundData.csvのデータフレームのサブセット
df = prepare_data.make_label_df_NoNA(csv_path, img_dir, img_suffix=img_suffix)
print('df.shape:', df.shape)
df.head()

df.shape: (2950, 14)


/home/tmp10014/.conda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,ID,set,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
7295,../../storage/img/Wash/331x331/train_all/NCGC0...,training,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7458,../../storage/img/Wash/331x331/train_all/NCGC0...,training,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7472,../../storage/img/Wash/331x331/train_all/NCGC0...,training,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7798,../../storage/img/Wash/331x331/train_all/NCGC0...,training,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7872,../../storage/img/Wash/331x331/train_all/NCGC0...,training,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_files, validation_files, test_files, y_train, y_valid, y_test = prepare_data.make_train_val_test(df)

print('train_files[0:5]:\n', train_files[0:5])
print('validation_files[0:5]:\n', validation_files[0:5])
print('test_files[0:5]:\n', test_files[0:5])
print('y_train[0:5]:\n', y_train[0:5])
print('y_valid[0:5]:\n', y_valid[0:5])
print('y_test[0:5]:\n', y_test[0:5])

training_set_y.shape: (2543, 12)
validation_set_y.shape: (125, 12)
test_set_y.shape: (282, 12)
train_files[0:5]:
 7295    ../../storage/img/Wash/331x331/train_all/NCGC0...
7458    ../../storage/img/Wash/331x331/train_all/NCGC0...
7472    ../../storage/img/Wash/331x331/train_all/NCGC0...
7798    ../../storage/img/Wash/331x331/train_all/NCGC0...
7872    ../../storage/img/Wash/331x331/train_all/NCGC0...
Name: ID, dtype: object
validation_files[0:5]:
 11767    ../../storage/img/Wash/331x331/test/NCGC002612...
11769    ../../storage/img/Wash/331x331/test/NCGC002615...
11771    ../../storage/img/Wash/331x331/test/NCGC002613...
11772    ../../storage/img/Wash/331x331/test/NCGC002616...
11774    ../../storage/img/Wash/331x331/test/NCGC002613...
Name: ID, dtype: object
test_files[0:5]:
 12075    ../../storage/img/Wash/331x331/Final_Evaluatio...
12087    ../../storage/img/Wash/331x331/Final_Evaluatio...
12088    ../../storage/img/Wash/331x331/Final_Evaluatio...
12090    ../../storage/img/Wash/33

## prepare_data.make_label_df_angle

In [8]:
# ラベル情報などが書いたcsvファイル（tox21_compoundData.csv）
csv_path = r'../../storage/tox21_compoundData.csv'
# IDにファイルのフルパスつける
img_dir = r'../../storage/img/Wash_060/331x331/'
# 画像ファイルのサフィックス
angle_suffix_list=['_060.jpg', '_120.jpg', '_180.jpg', '_240.jpg', '_300.jpg']
# 欠損値を-1に置換した回転用のtox21_compoundData.csvのデータフレーム
df = prepare_data.make_label_df_angle(csv_path, img_dir, angle_suffix_list=angle_suffix_list)
print(df.shape)
print(df['ID'][76242//6-1])
print(df.iloc[76242//6-1])

print(df['ID'][(76242//6)*2-1])
print(df.iloc[(76242//6)*2-1])

print(df['ID'][(76242//6)*3-1])
print(df.iloc[(76242//6)*3-1])

(76242, 19)
../../storage/img/Wash_060/331x331/Final_Evaluation/NCGC00357144-01/NCGC00357144-01_000.jpg
index                                                        12706
ID               ../../storage/img/Wash_060/331x331/Final_Evalu...
inchikey                               ZFFTZDQKIXPDAF-UHFFFAOYSA-N
sdftitle                                              NCGC00357144
order                                                        12706
set                                                           test
CVfold                                                          -1
NR.AhR                                                           0
NR.AR                                                            0
NR.AR.LBD                                                        0
NR.Aromatase                                                     0
NR.ER                                                            0
NR.ER.LBD                                                       -1
NR.PPAR.gamma            

## prepare_data.make_train_val_test_angle

In [9]:
# 回転用のTox21の画像のパスとラベル(y_train,y_valid,y_test)を取得する
train_files, validation_files, test_files, y_train, y_valid, y_test = prepare_data.make_train_val_test_angle(df)

print('train_files[0:5]:\n', train_files[0:5])
print('validation_files[0:5]:\n', validation_files[0:5])
print('test_files[0:5]:\n', test_files[0:5])
print('y_train[0:5]:\n', y_train[0:5])
print('y_valid[0:5]:\n', y_valid[0:5])
print('y_test[0:5]:\n', y_test[0:5])

training_set_y.shape: (70584, 12)
validation_set_y.shape: (296, 12)
test_set_y.shape: (647, 12)
train_files[0:5]:
 0    ../../storage/img/Wash_060/331x331/train_all/N...
1    ../../storage/img/Wash_060/331x331/train_all/N...
2    ../../storage/img/Wash_060/331x331/train_all/N...
3    ../../storage/img/Wash_060/331x331/train_all/N...
4    ../../storage/img/Wash_060/331x331/train_all/N...
Name: ID, dtype: object
validation_files[0:5]:
 11764    ../../storage/img/Wash_060/331x331/test/NCGC00...
11765    ../../storage/img/Wash_060/331x331/test/NCGC00...
11766    ../../storage/img/Wash_060/331x331/test/NCGC00...
11767    ../../storage/img/Wash_060/331x331/test/NCGC00...
11768    ../../storage/img/Wash_060/331x331/test/NCGC00...
Name: ID, dtype: object
test_files[0:5]:
 12060    ../../storage/img/Wash_060/331x331/Final_Evalu...
12061    ../../storage/img/Wash_060/331x331/Final_Evalu...
12062    ../../storage/img/Wash_060/331x331/Final_Evalu...
12063    ../../storage/img/Wash_060/331x331/Fina